# **Chapter 20: API Fundamentals**

---

## **20.1 What is an API?**

### **Defining Application Programming Interfaces**

An **Application Programming Interface (API)** is a set of protocols, routines, and tools that allows different software applications to communicate with each other. Think of it as a **contract** between systems: it defines how requests should be made, what data formats to use, and what responses to expect.

**Real-World Analogy:** 
Imagine a restaurant. The **menu** is the API documentation—it lists what you can order (available endpoints) and what you'll receive (responses). The **kitchen** is the server/backend. You (the client) don't need to know how the kitchen works; you just use the menu (API) to place your order (request) and receive food (response).

**Formal Definition:**
> "An API is an interface or communication protocol between a client and a server intended to simplify the building of client-side software." — Wikipedia

### **Why APIs Matter for Testing**

1. **Backend Validation:** UI tests verify the frontend; API tests verify the backend logic directly
2. **Speed:** API tests execute 10-100x faster than UI tests
3. **Stability:** No DOM changes, CSS flakiness, or JavaScript async issues
4. **Early Testing:** Test APIs before UI is built (shift-left)
5. **Integration Testing:** Verify communication between microservices
6. **Security:** Test authentication and authorization at the API layer

```
Testing Pyramid - API Position:

       /\
      /  \  E2E Tests (UI) - Slow, expensive
     /----\ 
    / API  \  Integration Tests - Medium speed
   /--------\
  /   Unit   \  Unit Tests - Fast, cheap
 /------------\
```

---

## **20.2 API Types and Architectures**

### **20.2.1 REST (Representational State Transfer)**

**REST** is the most common architectural style for web APIs. It's stateless, client-server based, and uses standard HTTP methods.

**Key Principles:**
- **Stateless:** Each request contains all information needed; server doesn't store client state
- **Client-Server:** Separation of concerns; UI and backend evolve independently  
- **Cacheable:** Responses can be cached by clients
- **Uniform Interface:** Standard HTTP methods and resource-based URLs
- **Layered System:** Client doesn't know if connected to end server or intermediary

**REST Example:**
```http
GET https://api.example.com/users/123
Accept: application/json

Response:
HTTP/1.1 200 OK
Content-Type: application/json

{
  "id": 123,
  "name": "John Doe",
  "email": "john@example.com",
  "links": {
    "self": "/users/123",
    "orders": "/users/123/orders"
  }
}
```

### **20.2.2 SOAP (Simple Object Access Protocol)**

**SOAP** is a protocol-based API architecture that uses XML for message formatting. It's more rigid and heavyweight than REST, commonly used in enterprise/financial systems.

**Characteristics:**
- **Protocol:** Strict rules and standards (WSDL, WS-Security)
- **Format:** XML only
- **Transport:** HTTP, SMTP, TCP, etc.
- **Security:** Built-in standards (WS-Security, encryption)
- **Reliability:** ACID compliance, transaction support

**SOAP Example:**
```xml
POST /UserService HTTP/1.1
Host: api.example.com
Content-Type: text/xml; charset=utf-8
SOAPAction: "http://example.com/GetUser"

<?xml version="1.0"?>
<soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope">
  <soap:Header>
    <AuthToken>abc123</AuthToken>
  </soap:Header>
  <soap:Body>
    <GetUser xmlns="http://example.com/users">
      <UserId>123</UserId>
    </GetUser>
  </soap:Body>
</soap:Envelope>
```

### **20.2.3 GraphQL**

**GraphQL** is a query language for APIs developed by Facebook. Unlike REST (where endpoints return fixed data structures), GraphQL allows clients to request exactly what they need.

**Key Features:**
- **Single Endpoint:** Usually `/graphql` for all operations
- **Flexible Queries:** Client specifies fields to return
- **Type System:** Strongly typed schema
- **Introspection:** API self-documenting via schema queries

**GraphQL Example:**
```http
POST /graphql HTTP/1.1
Content-Type: application/json

{
  "query": "query GetUser($id: ID!) { user(id: $id) { name email orders { total } } }",
  "variables": { "id": "123" }
}

Response:
{
  "data": {
    "user": {
      "name": "John Doe",
      "email": "john@example.com",
      "orders": [
        { "total": 99.99 },
        { "total": 45.50 }
      ]
    }
  }
}
```

### **Comparison: REST vs SOAP vs GraphQL**

| **Feature** | **REST** | **SOAP** | **GraphQL** |
|-------------|----------|----------|-------------|
| **Format** | JSON, XML, HTML | XML only | JSON |
| **Protocol** | HTTP | HTTP, SMTP, TCP, etc. | HTTP |
| **Flexibility** | High | Low (rigid standards) | Very High |
| **Caching** | HTTP caching | Custom | Application-level |
| **Learning Curve** | Low | High | Medium |
| **Security** | HTTPS, OAuth, JWT | WS-Security, SSL | HTTPS, OAuth |
| **Use Case** | Web services, Mobile | Banking, Enterprise | Complex data relationships |
| **Bandwidth** | Medium | High (XML overhead) | Low (precise queries) |
| **Tooling** | Postman, curl, etc. | SOAP UI | GraphiQL, Playground |

---

## **20.3 HTTP Methods and Status Codes (Deep Dive)**

### **HTTP Methods in Detail**

While Chapter 15 introduced HTTP basics, API testing requires deeper understanding of method semantics.

#### **GET - Retrieve Resources**
```python
import requests

# Simple GET
response = requests.get("https://api.example.com/users")

# GET with query parameters
params = {
    "page": 1,
    "limit": 10,
    "sort": "name",
    "order": "asc"
}
response = requests.get("https://api.example.com/users", params=params)

# The URL becomes: https://api.example.com/users?page=1&limit=10&sort=name&order=asc

# GET with headers
headers = {
    "Accept": "application/json",
    "Authorization": "Bearer token123"
}
response = requests.get("https://api.example.com/users/123", headers=headers)

# Testing considerations:
# - Idempotent: Same request produces same result
# - Safe: Should not modify server state
# - Cacheable: Responses can be cached
```

#### **POST - Create Resources**
```python
# POST with JSON body
new_user = {
    "name": "Jane Doe",
    "email": "jane@example.com",
    "role": "user"
}

response = requests.post(
    "https://api.example.com/users",
    json=new_user,  # Automatically sets Content-Type: application/json
    headers={"Authorization": "Bearer token123"}
)

# POST with form data (traditional HTML forms)
form_data = {
    "username": "jane",
    "password": "secret123"
}
response = requests.post(
    "https://api.example.com/login",
    data=form_data  # Content-Type: application/x-www-form-urlencoded
)

# POST with multipart (file upload)
files = {
    "file": ("report.pdf", open("report.pdf", "rb"), "application/pdf")
}
response = requests.post(
    "https://api.example.com/documents",
    files=files,
    data={"description": "Monthly report"}
)

# Testing considerations:
# - Not idempotent: Creating same resource twice creates duplicates
# - Returns 201 Created on success
# - Usually returns Location header with new resource URL
```

#### **PUT vs PATCH - Updates**

**PUT:** Full resource replacement (idempotent)
```python
# PUT - Replace entire resource
full_update = {
    "id": 123,
    "name": "John Updated",
    "email": "new@example.com",
    "role": "admin",
    "status": "active",
    "last_login": "2024-01-15"
}

response = requests.put(
    "https://api.example.com/users/123",
    json=full_update
)
# If any field missing, it might be set to null or default
```

**PATCH:** Partial update (not necessarily idempotent)
```python
# PATCH - Update only specified fields
partial_update = {
    "email": "newemail@example.com"
}

response = requests.patch(
    "https://api.example.com/users/123",
    json=partial_update
)
# Only email changes; other fields remain as before
```

#### **DELETE - Remove Resources**
```python
# Simple delete
response = requests.delete("https://api.example.com/users/123")

# Delete with body (some APIs require confirmation data)
response = requests.delete(
    "https://api.example.com/users/123",
    json={"reason": "user_request", "confirm": True}
)

# Testing considerations:
# - Idempotent: Deleting same resource twice should return 404 or 200
# - Returns 204 No Content on success (or 200 with deleted object)
```

#### **HEAD and OPTIONS**

```python
# HEAD - Get metadata without body (check existence, headers)
response = requests.head("https://api.example.com/users/123")
print(response.status_code)  # 200 = exists, 404 = not found
print(response.headers["Content-Length"])  # Size without downloading body

# OPTIONS - Check allowed methods (CORS preflight)
response = requests.options("https://api.example.com/users")
print(response.headers["Allow"])  # GET, POST, HEAD, OPTIONS
```

### **HTTP Status Codes Reference**

#### **2xx Success**
| **Code** | **Meaning** | **When to Use** |
|----------|-------------|-----------------|
| **200 OK** | Standard success | GET, PUT, PATCH successful |
| **201 Created** | Resource created | POST successful |
| **204 No Content** | Success, no body | DELETE successful, empty list |
| **206 Partial Content** | Range request success | Large file downloads |

#### **3xx Redirection**
| **Code** | **Meaning** | **Testing Note** |
|----------|-------------|------------------|
| **301 Moved Permanently** | Resource relocated permanently | Update test URLs |
| **302 Found** | Temporary redirect | Follow with `allow_redirects=True` |
| **304 Not Modified** | Cache valid | Check ETag/Last-Modified headers |

#### **4xx Client Errors**
| **Code** | **Meaning** | **Common Causes** |
|----------|-------------|-------------------|
| **400 Bad Request** | Malformed syntax | Invalid JSON, missing required fields |
| **401 Unauthorized** | Authentication required | Missing/invalid token |
| **403 Forbidden** | No permission | Valid auth but insufficient rights |
| **404 Not Found** | Resource doesn't exist | Wrong ID, deleted resource |
| **405 Method Not Allowed** | HTTP method not supported | POST to read-only endpoint |
| **409 Conflict** | Resource conflict | Duplicate entry, version mismatch |
| **422 Unprocessable Entity** | Validation failed | Semantic errors (valid JSON but bad values) |
| **429 Too Many Requests** | Rate limit exceeded | Add delays between requests |

#### **5xx Server Errors**
| **Code** | **Meaning** | **Action** |
|----------|-------------|------------|
| **500 Internal Server Error** | Generic server error | Report bug, retry may not help |
| **502 Bad Gateway** | Upstream error | Server behind proxy is down |
| **503 Service Unavailable** | Overload/maintenance | Retry with backoff |
| **504 Gateway Timeout** | Upstream timeout | Server took too long to respond |

---

## **20.4 Request and Response Structure**

### **HTTP Request Components**

```python
import requests

# Complete request with all components
response = requests.post(
    # URL (Uniform Resource Locator)
    url="https://api.example.com:443/v1/users?page=1",
    
    # Method specified by function (post, get, put, etc.)
    
    # Headers (Metadata)
    headers={
        "Content-Type": "application/json",      # Body format
        "Accept": "application/json",            # Expected response format
        "Authorization": "Bearer eyJhbGci...",   # Authentication
        "X-Request-ID": "uuid-123",              # Request tracing
        "X-API-Version": "v1",                   # API versioning
        "User-Agent": "MyTestClient/1.0"         # Client identification
    },
    
    # Query Parameters (URL)
    params={
        "page": 1,
        "limit": 10
    },
    
    # Request Body (for POST/PUT/PATCH)
    json={
        "name": "John Doe",
        "email": "john@example.com"
    },
    
    # Or form data
    # data={"key": "value"},
    
    # Or raw body
    # data='{"raw": "json"}',
    
    # Authentication shortcuts
    auth=("username", "password"),  # Basic Auth
    
    # Timeouts
    timeout=5,  # Seconds (connect timeout, read timeout)
    
    # SSL Verification
    verify=True,  # False only for testing with self-signed certs
    
    # Redirects
    allow_redirects=True
)
```

### **HTTP Response Components**

```python
# Analyzing the response
response = requests.get("https://api.example.com/users/123")

# Status Line
print(f"Status Code: {response.status_code}")  # 200
print(f"Reason: {response.reason}")            # OK

# Headers (Case-insensitive dictionary)
print(f"Content-Type: {response.headers['Content-Type']}")
print(f"Server: {response.headers.get('Server')}")
print(f"Date: {response.headers['Date']}")

# Response Body
print(f"Text: {response.text}")                # String format
print(f"JSON: {response.json()}")              # Parsed JSON (dict/list)
print(f"Content: {response.content}")          # Bytes (for binary)

# Encoding
print(f"Encoding: {response.encoding}")        # utf-8

# Cookies
print(f"Cookies: {response.cookies}")          # RequestsCookieJar

# URL (final after redirects)
print(f"Final URL: {response.url}")

# Request object (what was sent)
print(f"Request Method: {response.request.method}")
print(f"Request Headers: {response.request.headers}")
```

### **Content Types (MIME Types)**

| **Content-Type** | **Usage** | **Python Handling** |
|------------------|-----------|---------------------|
| `application/json` | JSON data | `response.json()` |
| `application/xml` | XML data | `response.text` + XML parser |
| `text/html` | HTML pages | `response.text` + BeautifulSoup |
| `text/plain` | Plain text | `response.text` |
| `multipart/form-data` | File uploads | `requests.post(files=...)` |
| `application/x-www-form-urlencoded` | Form submissions | `requests.post(data=...)` |
| `application/octet-stream` | Binary data | `response.content` (bytes) |

---

## **20.5 API Documentation Standards**

### **OpenAPI (Swagger) Specification**

**OpenAPI** is the standard format for describing REST APIs. It allows both humans and computers to understand the capabilities of a service without accessing the source code.

**Key Sections:**
- **Paths:** Available endpoints and operations (GET, POST, etc.)
- **Parameters:** Input parameters (query, path, header, body)
- **Responses:** Expected status codes and response schemas
- **Components:** Reusable schemas, parameters, security schemes

**Example OpenAPI 3.0 Snippet:**
```yaml
openapi: 3.0.0
info:
  title: User API
  version: 1.0.0

paths:
  /users/{id}:
    get:
      summary: Get user by ID
      parameters:
        - name: id
          in: path
          required: true
          schema:
            type: integer
      responses:
        '200':
          description: User found
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/User'
        '404':
          description: User not found

components:
  schemas:
    User:
      type: object
      properties:
        id:
          type: integer
        name:
          type: string
        email:
          type: string
          format: email
```

**Using OpenAPI for Testing:**

```python
# Parse OpenAPI spec to generate tests
import yaml
import requests

def load_openapi_spec(path):
    with open(path, 'r') as f:
        return yaml.safe_load(f)

def test_endpoints_from_spec():
    spec = load_openapi_spec('openapi.yaml')
    base_url = "https://api.example.com"
    
    for path, methods in spec['paths'].items():
        for method, details in methods.items():
            if method == 'get':
                # Test GET endpoint
                url = f"{base_url}{path}"
                response = requests.get(url)
                
                # Verify status code matches spec
                assert str(response.status_code) in details['responses']
                
                # Verify content type
                if '200' in details['responses']:
                    expected_types = details['responses']['200']['content']
                    assert response.headers['Content-Type'] in expected_types

# Alternative: Use schemathesis library for property-based testing from OpenAPI
```

### **Postman Collections**

**Postman Collections** are JSON files describing API requests that can be shared and executed.

**Structure:**
- **Info:** Collection metadata
- **Item:** Individual requests (can be nested in folders)
- **Request:** HTTP method, URL, headers, body
- **Event:** Pre-request and test scripts
- **Variable:** Environment variables

**Converting Postman to Python:**
```python
# Export Postman collection to JSON, then convert:

import json

def postman_to_python(collection_path):
    with open(collection_path) as f:
        collection = json.load(f)
    
    for item in collection['item']:
        name = item['name']
        request = item['request']
        
        method = request['method'].lower()
        url = request['url']['raw']
        headers = {h['key']: h['value'] for h in request.get('header', [])}
        
        # Generate Python code
        print(f"\n# {name}")
        print(f"response = requests.{method}(")
        print(f"    '{url}',")
        print(f"    headers={headers}")
        print(f")")
        print(f"assert response.status_code == {item['response'][0]['code']}")

# Or use Postman's "Code" feature to generate Python requests code directly
```

---

## **20.6 Writing Your First API Tests**

### **Basic API Testing Pattern**

```python
import requests
import pytest

class TestUserAPI:
    """Basic API testing patterns"""
    
    BASE_URL = "https://api.example.com/v1"
    HEADERS = {
        "Content-Type": "application/json",
        "Accept": "application/json"
    }
    
    def test_get_user_success(self):
        """Test successful retrieval of user"""
        # Arrange
        user_id = 1
        
        # Act
        response = requests.get(
            f"{self.BASE_URL}/users/{user_id}",
            headers=self.HEADERS
        )
        
        # Assert
        assert response.status_code == 200
        assert response.headers["Content-Type"] == "application/json"
        
        data = response.json()
        assert data["id"] == user_id
        assert "name" in data
        assert "email" in data
        assert "@" in data["email"]  # Basic email validation
    
    def test_get_user_not_found(self):
        """Test 404 for non-existent user"""
        response = requests.get(
            f"{self.BASE_URL}/users/999999",
            headers=self.HEADERS
        )
        
        assert response.status_code == 404
        data = response.json()
        assert "error" in data or "message" in data
    
    def test_create_user_success(self):
        """Test user creation"""
        # Arrange
        payload = {
            "name": "Test User",
            "email": f"test_{pytest.util.uuid.uuid4().hex[:8]}@example.com",
            "role": "user"
        }
        
        # Act
        response = requests.post(
            f"{self.BASE_URL}/users",
            json=payload,
            headers=self.HEADERS
        )
        
        # Assert
        assert response.status_code == 201
        
        # Verify Location header present
        assert "Location" in response.headers
        new_url = response.headers["Location"]
        assert "/users/" in new_url
        
        # Verify response body
        data = response.json()
        assert data["id"] is not None
        assert data["name"] == payload["name"]
        assert data["email"] == payload["email"]
        
        # Cleanup (if API supports delete)
        user_id = data["id"]
        requests.delete(f"{self.BASE_URL}/users/{user_id}")
    
    def test_update_user_partial(self):
        """Test PATCH for partial updates"""
        # Create test user first
        create_resp = requests.post(
            f"{self.BASE_URL}/users",
            json={"name": "Original", "email": "orig@example.com"},
            headers=self.HEADERS
        )
        user_id = create_resp.json()["id"]
        
        try:
            # Update only email
            patch_response = requests.patch(
                f"{self.BASE_URL}/users/{user_id}",
                json={"email": "updated@example.com"},
                headers=self.HEADERS
            )
            
            assert patch_response.status_code == 200
            
            # Verify update
            get_response = requests.get(f"{self.BASE_URL}/users/{user_id}")
            data = get_response.json()
            
            assert data["email"] == "updated@example.com"
            assert data["name"] == "Original"  # Unchanged
            
        finally:
            # Cleanup
            requests.delete(f"{self.BASE_URL}/users/{user_id}")
    
    def test_delete_user(self):
        """Test user deletion"""
        # Create user to delete
        create_resp = requests.post(
            f"{self.BASE_URL}/users",
            json={"name": "To Delete", "email": "delete@example.com"},
            headers=self.HEADERS
        )
        user_id = create_resp.json()["id"]
        
        # Delete
        delete_resp = requests.delete(
            f"{self.BASE_URL}/users/{user_id}",
            headers=self.HEADERS
        )
        
        assert delete_resp.status_code in [200, 204]
        
        # Verify deletion
        get_resp = requests.get(f"{self.BASE_URL}/users/{user_id}")
        assert get_resp.status_code == 404
    
    def test_response_schema_validation(self):
        """Validate response structure matches expected schema"""
        import jsonschema
        
        expected_schema = {
            "type": "object",
            "required": ["id", "name", "email"],
            "properties": {
                "id": {"type": "integer"},
                "name": {"type": "string", "minLength": 1},
                "email": {"type": "string", "format": "email"},
                "role": {"type": "string", "enum": ["admin", "user", "guest"]}
            }
        }
        
        response = requests.get(f"{self.BASE_URL}/users/1")
        data = response.json()
        
        # Validate against schema
        jsonschema.validate(instance=data, schema=expected_schema)
```

### **Testing API Contracts**

```python
class TestAPIContract:
    """Verify API adheres to its contract"""
    
    def test_header_contract(self):
        """Verify required headers present"""
        response = requests.get("https://api.example.com/users")
        
        # Security headers
        assert "X-Content-Type-Options" in response.headers
        assert response.headers["X-Content-Type-Options"] == "nosniff"
        
        # CORS headers (if applicable)
        if "Access-Control-Allow-Origin" in response.headers:
            assert response.headers["Access-Control-Allow-Origin"] in [
                "*", 
                "https://example.com"
            ]
    
    def test_error_response_format(self):
        """Verify error responses follow standard format"""
        response = requests.get("https://api.example.com/invalid-endpoint")
        
        assert response.status_code == 404
        
        data = response.json()
        
        # Standard error structure
        assert "error" in data or "errors" in data
        assert "message" in data or "detail" in data
        assert "code" in data or "status" in data
    
    def test_pagination_contract(self):
        """Verify pagination metadata"""
        response = requests.get(
            "https://api.example.com/users",
            params={"page": 2, "limit": 10}
        )
        
        data = response.json()
        
        # Common pagination patterns
        if isinstance(data, dict) and "data" in data:
            # Wrapper pattern: {"data": [...], "pagination": {...}}
            assert "pagination" in data or "meta" in data
            meta = data.get("pagination", data.get("meta", {}))
            assert "current_page" in meta or "page" in meta
            assert "total_pages" in meta or "total" in meta
        elif isinstance(data, list) and "Link" in response.headers:
            # Header-based pagination (GitHub style)
            link_header = response.headers["Link"]
            assert "rel=\"next\"" in link_header or "rel=\"prev\"" in link_header
```

---

## **Chapter Summary**

### **Key Takeaways:**

1. **API Definition:** APIs are contracts allowing software systems to communicate. They abstract implementation details, exposing only necessary functionality.

2. **API Types:**
   - **REST:** Resource-based, HTTP methods, JSON/XML, stateless (most common)
   - **SOAP:** Protocol-based, XML only, strict standards, enterprise-focused
   - **GraphQL:** Query language, single endpoint, flexible data retrieval, strongly typed

3. **HTTP Methods:**
   - **GET:** Retrieve (safe, idempotent)
   - **POST:** Create (not idempotent, returns 201)
   - **PUT:** Full update (idempotent, replace entire resource)
   - **PATCH:** Partial update (modify specific fields)
   - **DELETE:** Remove (idempotent)
   - **HEAD/OPTIONS:** Metadata inspection

4. **Status Codes:** 2xx (success), 3xx (redirect), 4xx (client error), 5xx (server error). Each code has specific semantics for API behavior.

5. **Request/Response:** Headers carry metadata (auth, content type), body carries payload. Content-Type and Accept headers negotiate data formats.

6. **Documentation:** OpenAPI (Swagger) provides machine-readable API specs. Postman collections offer executable documentation. Both enable test generation.

7. **Testing Approach:** Verify status codes, response schemas, headers, and business logic. Treat APIs as contracts—test that they adhere to their specification.

### **Best Practices:**

- **State Independence:** Each test should setup and teardown its data
- **Schema Validation:** Validate responses against JSON schemas
- **Header Verification:** Check Content-Type, security headers, and custom headers
- **Error Testing:** Verify 4xx and 5xx responses are handled gracefully
- **Cleanup:** Always delete test data after creation
- **Idempotency:** Verify PUT/DELETE are idempotent (same result on retry)

---

## **📖 Next Chapter: Chapter 21 - REST API Testing**

Now that you understand API fundamentals, **Chapter 21** dives deep into **REST API Testing**—the most common API architecture in modern web development.

In **Chapter 21**, you'll master:

- **REST API Architecture:** Resources, URIs, representations, and statelessness in depth
- **Authentication Mechanisms:** Basic Auth, Bearer tokens, OAuth 2.0, API Keys, JWT handling
- **Advanced Request Techniques:** Query parameters, path variables, headers, and body formats
- **Response Validation:** JSON schema validation, XML parsing, and data extraction
- **Testing Tools:** REST Assured (Java), requests (Python), Postman scripting
- **API Test Automation:** Data-driven testing, environment management, and CI integration
- **Common API Patterns:** Pagination, filtering, sorting, rate limiting, and versioning

**Why Chapter 21 is Critical:** REST APIs power the modern web. Mobile apps, SPAs, and microservices all consume REST APIs. Testing these APIs thoroughly ensures your backend logic is solid before the UI is even built. This chapter provides the practical skills to automate REST API testing professionally.

**Continue to Chapter 21 to become an expert in REST API test automation!**